# LangChain: Agents

Some applications require not just a predetermined chain of calls to LLMs/other tools, but potentially an unknown chain that depends on the user’s input. In these types of chains, there is an agent which has access to a suite of tools. Depending on the user input, the agent can then decide which, if any, of these tools to call.

> You can think of a large language model as a reasoning engine, in which you can give it chunks of text or other sources of information. Then, the LLM can use that information (in addition to its background knowledge that's learned off the internet) to answer questions or reason through content or decide even what to do next. And that's what LangChain's Agents framework helps you to do.

In [1]:
import openai
from dotenv import load_dotenv, find_dotenv
import os

_ = load_dotenv(find_dotenv())  # add .env to .gitignore
openai.api_key = os.getenv("OPENAI_API_KEY")

import warnings
warnings.filterwarnings("ignore")

## Built-in LangChain tools

In [2]:
# %%capture
# %pip install -U wikipedia

In [3]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

Setting the temperature equal to zero is important because we're going to be using the LLM as the reasoning engine of an agent, where it's connecting to other sources of data and computation.

And so we want this reasoning engine to be as good and as precise as possible and so we're going to set it to zero to get rid of any randomness that might arise.


In [4]:
llm = ChatOpenAI(temperature=0)

Next, we're going to load the LLM Math tool and the Wikipedia tool.


The LLM Math tool is actually a chain itself, which uses a language model in conjunction with a calculator to do math problems. The Wikipedia tool is an API that connects to Wikipedia, allowing you to run search queries against Wikipedia and get back results.

In [5]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

We're going to initialize the agent with the following parameters:
* tools 
* the language model 
* agent type: Here, we're going to use chat zero-shot React description. The important things to note here are first, **CHAT**, This is an agent that has been optimized to work with chat models. And second, **REACT**, This is a prompting technique designed to get the best reasoning performance out of language models.
* ``handleParsingErrors`` equals true. This is useful when the language model might output something that is not able to be parsed into an action and action input, which is the desired output. When this happens, we'll actually pass the misformatted text back to the language model and ask it to correct itself.
* ``verbose`` equals true. Prints out a bunch of steps that makes it really clear to us in the Jupyter Notebook what's going on.

In [6]:
agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

### Math tool

Let's ask the agent a simple math question to understand how it works.

In [7]:
agent("What is the 25% of 300?")



> Entering new AgentExecutor chain...
I can use the calculator tool to find the answer to this question.

Action:
```json
{
  "action": "Calculator",
  "action_input": "25% of 300"
}
```
Observation: Answer: 75.0
Thought:The answer is 75.0.
Final Answer: 75.0

> Finished chain.


{'input': 'What is the 25% of 300?', 'output': '75.0'}

We can see above that when it enters the ``AgentExecutor chain``, that it first thinks about what it needs to do. 

So it has a ``Thought``. It then has an ``Action``, and this action is actually a JSON blob corresponding to two things, an ``action`` and an ``action input``: 

* ``action`` corresponds to the tool to use. So here it says calculator. 
* ``action input`` is the input to that tool. Here it's a string of 300 times 0.25. 

Next, we can see that there's ``Observation`` with answer in a **separate color**. This observation answer equals 75.0 is actually coming from the **calculator** tool itself.

Next, we go back to the LLM when the text turns to green (``Thought``). We have the answer to the question.

Final answer, 75.0, and that's the output that we get.

> NOTE: Different colors are used in the output to denote different tools.

In [8]:
# Use visualization tool 
from langchain_visualizer import visualize, visualize_embeddings

async def run_chain():
    return agent("What is the 25% of 300?")

visualize_embeddings()
visualize(run_chain)

2023-07-28 09:44.33.094179 [info     ] Trace: http://127.0.0.1:8935/traces/01H6DQ9NT55Z7FVMR98MK9SAV2


> Entering new AgentExecutor chain...
I can use the calculator tool to find the answer to this question.

Action:
```json
{
  "action": "Calculator",
  "action_input": "25% of 300"
}
```2023-07-28 09:44.35.338825 [info     ] Starting server, set OUGHT_ICE_AUTO_SERVER=0 to disable.

Observation: Answer: 75.0
Thought:2023-07-28 09:44.36.889774 [info     ] Server started! Run `python -m ice.server stop` to stop it.
2023-07-28 09:44.36.891686 [info     ] Opening trace in browser, set OUGHT_ICE_AUTO_BROWSER=0 to disable.
The answer is 75.0.
Final Answer: 75.0

> Finished chain.


{'input': 'What is the 25% of 300?', 'output': '75.0'}

### Wikipedia tool
Next, we're going to go through an example using the Wikipedia API. Here, we are asking a question about Tom Mitchell, and we can look at the intermediate steps to see what it does.

In [8]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
result = agent(question) 



> Entering new AgentExecutor chain...
Thought: I should use Wikipedia to find the answer to this question.

Action:
```
{
  "action": "Wikipedia",
  "action_input": "Tom M. Mitchell"
}
```


Observation: Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former Chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science and a Fellow and past President of the Association for the Advancement of Artificial Intelligence. In October 2018, Mitchell was appointed as the Int

We can see once again that it thinks and it correctly realizes that it should use Wikipedia.

It says ``action`` equal to Wikipedia and ``action input`` equal to Tom M. Mitchell.

The ``Observation`` that comes back (in yellow this time,) is the Wikipedia summary result for the Tom M. Mitchell page.

The ``Observation`` that comes back from Wikipedia is actually two results, two pages as there's two different Tom M. Mitchells (the computer scientist and an Australian footballer)

We can see that the information needed to answer this question, namely the name of the book that he wrote (Machine Learning) is present in the summary of the first Tom M. Mitchell.

We can see next that the agent tries to look up more information about this book.

It looks up Machine Learning book in Wikipedia. This isn't strictly necessary, and it's an interesting example to show how agents aren't perfectly reliable yet.

We can see that after this lookup, the agent recognizes that it has all the information it needs to answer, and responds with the correct answer, Machine Learning.

## Python Agent

If you've seen things like Copilot or even ChatGPT with the code interpreter plugin enabled, one of the things they're doing is they're using the language model to write code, and then executing that code. We can do the same exact thing here. 

We're going to create a Python agent, and we're going to use the same LLM as before, and we're going to give it a tool, the Python REPL tool. A REPL is basically a way to interact with code. You can think of it as a Jupyter Notebook. The agent can execute code with this REPL. It will then run, and then we'll get back some results. 

Those results will be passed back into the agent, so it can decide what to do next.

In [10]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

The problem that we're going to have this agent solve is we're going to give it a list of names, and then ask it to sort them. 

In [11]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

Importantly, we're asking it to print the output, so that it can actually see what the result is. These printed statements are what's going to be fed back into the language model later on, so it can reason about the output of the code that it just ran. Let's give this a try. 

In [10]:
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 



> Entering new AgentExecutor chain...
I can use the sorted() function to sort the list of customers by last name and then first name. I will need to provide a key function to sorted() that returns a tuple of the last name and first name in that order.
Action: Python REPL
Action Input:
```
customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]
sorted_customers = sorted(customers, key=lambda x: (x[1], x[0]))
for customer in sorted_customers:
    print(customer)
```
Observation: ['Jen', 'Ayai']
['Lang', 'Chain']
['Harrison', 'Chase']
['Elle', 'Elem']
['Trance', 'Former']
['Geoff', 'Fusion']
['Dolly', 'Too']

Thought:The customers are now sorted by last name and then first name. 
Final Answer: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]

> Finished chain.


"[['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]"

We can see that when we go into the ``AgentExecutor chain``, it first realizes that it can use the sorted function to list the customers. It's using a different agent type under the hood, which is why you can see that the ``action`` and ``action input`` is actually formatted slightly differently. 

Here, the ``action`` that it takes is to use the Python ``REPL``, and then the ``action input`` that you can see is code, where it first writes out customers equals this list, it then sorts the customers, and then it goes through this list and print it. 

You can see the agent thinks about what to do and realizes that it needs to write some code. The format that it's using of ``action`` and ``action input`` is actually slightly different than before. It's using a **different agent type** under the hood. For the ``action``, it's going to use the Python ``REPL``, and for the action input, it's going to have a bunch of code. 

If we look at what this code is doing, it's first creating a variable to list out these customer names. It's then sorting that and creating a new variable, and it's then iterating through that new variable and printing out each line, just like we asked it to. 

We can see that we get the ``Observation`` back, and this is a list of names, and then the agent realizes that it's done and it returns these names. We can see from the stuff that's printed out the high level of what's going on.

#### View detailed outputs of the chains

Let's dig a little bit deeper and prints out all the levels of all the different chains.

In [12]:
import langchain
langchain.debug=True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 
langchain.debug=False

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nI

1. First, we start with the ``[chain/start] [1:chain:AgentExecutor]``. This is the top level agent runner, and we can see that we have here our input.

2. From ``[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain]``, we call an ``LLM chain`` that the agent is using. At this point, it's only got the ``input``, ``agent scratchpad``. and some ``stop`` sequences to tell the language model when to stop doing its generations.

> Remember: **LLM chain** is a combination of prompt and an LLM.

3. At the next level `[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI]`, we see the exact call to the language model. We can see the fully formatted ``prompt``, which includes instructions about what tools it has access to, as well as how to format its output.

4. From ``[llm/end] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI]``, we can then see the exact output of the language model. We can see the ``text`` key where it has the thought and the action and the action input all in one string.

5. `[chain/end] [1:chain:AgentExecutor > 2:chain:LLMChain]` It then wraps up the LLM chain as it exits through there.
---
6. `[tool/start] [1:chain:AgentExecutor > 4:tool:Python REPL]`: The next thing that it calls is a tool. Here, we can see the exact input to the tool (Python REPL) Then we can see the input, which is this code.

7. `[tool/end] [1:chain:AgentExecutor > 4:tool:Python REPL]` We can then see the output of this tool, which is this printed out string, because we've specifically asked the Python REPL to print out what is going on.

8. `[chain/start] [1:chain:AgentExecutor > 5:chain:LLMChain]` We can then see the next input to the LLM chain, which again, the LLM chain here is the agent. Here, if you look at the variables, the ``input`` is unchanged. It is the high-level objective that we're asking. But now there's some new values for ``agent scratchpad``, which is actually a combination of the previous generation plus the tool output We're passing ``agent scratchpad`` back in so that the language model can understand what happened previously and use that to reason about what to do next.

The next few print statements are covering what happens as the language model realizes that it is basically finished with its job:

9. ``[llm/start] [1:chain:AgentExecutor > 5:chain:LLMChain > 6:llm:ChatOpenAI]``: We can see here the fully formatted prompt to the language model, 
10. ``[llm/end] [1:chain:AgentExecutor > 5:chain:LLMChain > 6:llm:ChatOpenAI]``: the response where it realizes that it is done, and it says final answer, which here is the sequence that the agent uses to recognize that it's done with its job.

11. We can then see it exiting the LLM chain `[chain/end] [1:chain:AgentExecutor > 5:chain:LLMChain]` and then exiting the agent executor `[chain/end] [1:chain:AgentExecutor]`.

This should hopefully give you a pretty good idea of what's going on under the hood inside these agents.


In [12]:
async def run_chain():
    return agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""")

visualize_embeddings()
visualize(run_chain)

2023-07-28 09:49.57.696752 [info     ] Trace: http://127.0.0.1:8935/traces/01H6DQKJSZAZ3D6WY3GYBACZRV


> Entering new AgentExecutor chain...
2023-07-28 09:49.57.731538 [info     ] Opening trace in browser, set OUGHT_ICE_AUTO_BROWSER=0 to disable.
I can use the `sorted()` function to sort the list of customers. I will need to provide a key function that specifies the sorting order based on last name and then first name.
Action: Python_REPL
Action Input: sorted([['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']], key=lambda x: (x[1], x[0]))

Python REPL can execute arbitrary code. Use with caution.



Observation: 
Thought:The customers have been sorted by last name and then first name.
Final Answer: [['Jen', 'Ayai'], ['Harrison', 'Chase'], ['Lang', 'Chain'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Dolly', 'Too']]

> Finished chain.


[['Jen', 'Ayai'], ['Harrison', 'Chase'], ['Lang', 'Chain'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Dolly', 'Too']]

## Define your own tool

The big power of agents is that you can connect it to your own sources of information, your own APIs, your own data. That is, you can create a custom tool so that you can connect it to whatever you want.

Let's make a tool that's going to tell us what the current date is.

In [13]:
# %%capture
# %pip install DateTime

First, we're going to import this ``tool`` decorator. This can be applied to ANY function, and it turns it into a tool that LangChain can use.

In [14]:
from langchain.agents import tool
from datetime import date

Next, we're going to write a function called ``time``, which takes in any text string and returns today's date by calling dateTime.

In addition to the name of the function, we're also going to write a really detailed doc string. That's because this is what the ``agent`` will use to know when it should call this tool and how it should call this tool.

For example, here we say that the input should always be an empty string. That's because we don't use it. If we have more string requirements on what the input should be, for example, if we have a function that should always take in a search query or a SQL statement, you'll want to make sure to mention that here.

In [15]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

We're now going to create another ``agent``. This time we're adding the time tool to the list of existing tools.

In [16]:
agent= initialize_agent(
    tools + [time], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

And finally, let's call the ``agent`` and ask it what the date today is:

> **Note**: The agent will sometimes come to the wrong conclusion (agents are a work in progress!). If it does, please try running it again.

In [17]:
try:
    result = agent("whats the date today?") 
except: 
    print("exception on external access")



> Entering new AgentExecutor chain...
Thought: I need to use the `time` tool to get today's date.
Action:
```
{
  "action": "time",
  "action_input": ""
}
```

Observation: 2023-06-13
Thought:I have successfully retrieved today's date using the `time` tool.
Final Answer: Today's date is 2023-06-13.

> Finished chain.


 The `agent` recognizes that it needs to use the ``time`` tool, which it specifies above. It has the ``action input`` as an empty string. This is great. This is what we told it to do. Then, it returns with an ``Observation``.